In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import csv
# for parallel processing
from multiprocess import Pool
from scipy.sparse.csgraph import shortest_path

# import plotting functions
from harvey_plotting_v53 import *

# import loop extrusion codes
import pyximport
pyximport.install(setup_args={'include_dirs':np.get_include()})
from loop_extrusion_twoLEFpopulations_CollidingSuperExtruder_SuperLoading_v15 import LEFTranslocatorDirectional

## Set up the chromosome

In [ ]:
# define output folder 
output_folder = 'output_1013_test_run1'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

tiling_factor = 600 # number of TAD arrays repeated

# set up boundary elements
boundary_coordinates = [0, 200,201, 601,602, 802,803, 1603,1604, 1804,1805, 2205,2206, 2406,2407, 3607] # coordinates of boundary elements
TAD_array_size =  boundary_coordinates[-1] + 1 # the size of the big TAD in kb
chrom_size = TAD_array_size*tiling_factor # the total size of the chromosome in kb
boundary_coordinates_o = boundary_coordinates # coordinates of boundary elements within the first TAD array

# obtain the coordinates of all boundary elements and the double strand break sites on the chromosome
for j in range(tiling_factor-1):
    k = TAD_array_size*(j+1)
    res = [x + k for x in boundary_coordinates_o] 
    boundary_coordinates = boundary_coordinates + res
    
# randomly determine the coordinate of the DSB in the first TAD
random_pick_0 = int(np.random.random()*(boundary_coordinates_o[1]-boundary_coordinates_o[0]-2)+boundary_coordinates_o[0]+1)
if random_pick_0+1 == boundary_coordinates[1]:
    DSB_coordinates = [random_pick_0-1, random_pick_0]
else:
    DSB_coordinates = [random_pick_0, random_pick_0+1]

# counters for the number of subTADs with a double strand break
tad_200 = 1 
tad_400 = 0
tad_800 = 0
tad_1200 = 0
tad_count = np.asarray([1,0,0,0]) # counter for TAD of each size containing DSB
tad_size = np.asarray([200, 400, 800, 1200]) 


dist_between_break = 10000 #kb distances between two DSB

DSB_dist_to_CTCF = [min(DSB_coordinates[0]-boundary_coordinates[0],boundary_coordinates[1]-DSB_coordinates[0])] 

DSB_TAD_array_size = [200]
within_tile = True
while within_tile:
    # skip dist_between_break 
    next_break = DSB_coordinates[-1] + dist_between_break
    # preventing the next DSB occurring at a boundary element
    if np.count_nonzero(next_break - np.asarray(boundary_coordinates))<len(boundary_coordinates):
        next_break += 2
    # determine the TAD the next DSB occurs in
    test = np.asarray(boundary_coordinates) - next_break
    boundray_index = np.argmax(test>0) # the first BE coordinates to the right of the DSB
    # randomize the coordinate of the DSB within the TAD (so that the distance between DSB and BE is randomized)
    random_pick = int(np.random.random()*(boundary_coordinates[boundray_index]-boundary_coordinates[boundray_index-1]-2)+boundary_coordinates[boundray_index-1]+1)
    if random_pick+1 == boundary_coordinates[boundray_index]:
        DSB_coordinates += [random_pick-1, random_pick]
    else:
        DSB_coordinates += [random_pick, random_pick+1]
    tad_size_index = np.argwhere(tad_size==boundary_coordinates[boundray_index]-boundary_coordinates[boundray_index-1])[0][0]
    tad_count[tad_size_index]+=1
    DSB_TAD_array_size.append(tad_size[tad_size_index])
    if DSB_coordinates[-1] + dist_between_break> chrom_size-1:
        within_tile = False


DSB_TAD_array_size = np.asarray(DSB_TAD_array_size)
        
LogFileName = output_folder+"/log.txt"

o = open(LogFileName, "w") 
o.close()

with open(LogFileName, "a") as f:
    f.write('Tiling factor: \n')
    f.write(str(tiling_factor)+'\n')
    for i in range(len(tad_size)):
        f.write('number of breaks within a '+str(tad_size[i])+'kb TADs: \n')
        f.write(str(tad_count[i])+'\n')

file = output_folder+'/DSB_boundary_coordinates.npy' 

# saving DSB coordinates and boundary coordinates to robustness 
if not os.path.exists(file):
    with open(file, 'wb') as g:
        np.save(g, np.asarray(DSB_coordinates))
        np.save(g, np.asarray(boundary_coordinates))
    

## Set up the parameter sweep & simulation run

In [ ]:
## Define simulation parameters
# list of (separation,processivity) tuple 
sep_proc_list = []
sep_list = [125]
proc_list = [250]
for sep in sep_list:
    for proc in proc_list:
        sep_proc_list.append((sep,proc))
        
proc_ratio = 20 # the ratio of long-lived LEF processivity over normal LEF processivity  
BE_stabilization_factor_list =  [16] # the fold stabilization of LEF at BE
DSB_stabilization_factor_list = [4] # the fold stabilization of LEF at DSB ends
boundary_strength_list = [0.5] # the probability of LEF stalled by an boundary element
longlived_fraction_list = [0.2] # percentage of long-lived LEFs in the total LEF polulation
targetedloading_factor_list = [1000] # fold increase of loading probability at the DSB ends

alpha_list = [1, 3, 5]   # threshold distance: half of the unfilled gap size (in kb) for calling synapsis

# other input parameters
LEF_step_probability = 1 # the probability LEF taking a step
    
# maximum number of steps post DSB to stop simulation
step_count_limit = 40000

# whether to take snapshots of the synapsis process
plot_snapshots = False

In [ ]:
def do_one_parameter_set(sep_proc_pair, boundary_strength, BEstabilization_factor, DSBstabilization_factor,longlived_fraction, super_loading_factor, proc_ratio,chrom_size, step_count_limit,alpha_list,boundary_coordinates,DSB_coordinates,LEF_step_probability,DSB_TAD_array_size, plot_snapshots = False):
    
    # assume uniform loading probability on the whole chromosome
    loading_prob = np.ones(chrom_size)/chrom_size # the probability LEF loads at each lattice site on the chromosome
    stall_prob_left = np.zeros(chrom_size, dtype=np.double) # numpy array to store the probability of left motor subunits of LEFs being stalled
    stall_prob_right = np.zeros(chrom_size, dtype=np.double) # numpy array to store the probability of right motor subunits of LEFs being stalled
    
    separations = sep_proc_pair[0]
    processivity = sep_proc_pair[1]
    processivity_longlived = processivity * proc_ratio #processivity of long-lived LEFs
    
    # if boundary_strength is 0, there is no stabilization of LEF at BE
    BEstabilization_factor_o = BEstabilization_factor 
    if boundary_strength == 0:
        BEstabilization_factor = 1
        
    # list to store the first passage time
    first_passage = []
    # list to store the fraction of DSBs constrained by LEFs
    restrained_fraction = []
    # list to store fractions related to BE stabilization
    stabilized_fraction = []
    # list to store the size of last constraining LEF before it unloads for DSBs not synapsed
    fail_gap_length_l = [] 
    # list to store the size of inner most constraining LEF when synapsis is achieved
    success_gap_length_l = [] 
    
    # deathProb, the probability that LEF unloads from chromosome
    death_prob = np.zeros(chrom_size, dtype=np.double) + 1. / (0.5*processivity/(LEF_step_probability)) #times 0.5 to account for two-sided extrusion 
    death_prob_longlived = np.zeros(chrom_size, dtype=np.double) + 1. / (0.5*processivity_longlived/(LEF_step_probability)) # for super extruders
    
    # set the probability LEFs get stalled at lattice sites representing BE to be boundary strength
    stall_prob_left[boundary_coordinates[::2]] = boundary_strength # odd elements of the boundary coordinates
    stall_prob_right[boundary_coordinates[1::2]] = boundary_strength # even elements of the boundary coordinates
    
    # the probability that LEF does not take a step
    pause_prob = (1-LEF_step_probability)*np.ones(chrom_size, dtype=np.double) 

    # the probability that LEF unloads at BE reduces if there is stabilization by BE
    death_prob[boundary_coordinates] = 1./ (0.5*processivity/(LEF_step_probability))/ BEstabilization_factor
    death_prob_longlived[boundary_coordinates] = 1./ (0.5*processivity_longlived/(LEF_step_probability))/ BEstabilization_factor
    
    # set the separations for normal LEFs and long-lived LEFs so that the separation for all LEFs matches the input parameter separation
    normal_sep = separations/(1-longlived_fraction)
    num_smcs = int(np.round(chrom_size/normal_sep))
    if longlived_fraction==0:
        num_smcs_super = 0
    else:
        super_sep = separations/longlived_fraction
        num_smcs_super = int(np.round(chrom_size/super_sep))
    
    # create Boolean array for BE
    boundary_coordinate_array = np.asarray(boundary_coordinates)
    boundary_flag = np.zeros(chrom_size, int)
    boundary_flag[boundary_coordinate_array] = 1 
    
    # create simulation object with parameters defined above
    transloc = LEFTranslocatorDirectional(loading_prob, death_prob, death_prob_longlived, stall_prob_left, stall_prob_right,\
                                          pause_prob, num_smcs, boundary_flag,num_smcs_super) 
    
    # number of simulation steps prior to DSB to take snapshots
    preDSB_step_count_to_take_snapshots = 60
    # let the system reach steady state
    transloc.steps(100000-preDSB_step_count_to_take_snapshots)
    
    
    # get LEF coordinates
    super_left_extruders, super_right_extruders = transloc.getlonglivedLEFs()
    normal_left_extruders, normal_right_extruders = transloc.getLEFs()
    left_extruders  =  np.concatenate((super_left_extruders,normal_left_extruders))
    right_extruders  =  np.concatenate((super_right_extruders,normal_right_extruders))
    super_extruder_len = len(super_left_extruders)
    
    # lists to store gap sizes for plotting in snapshots
    gapsize_list_left = [[] for i in range(int(len(DSB_coordinates)/2)//10+1)]
    gapsize_list_right = [[] for i in range(int(len(DSB_coordinates)/2)//10+1)]
    # lists of initial constraining LEF coordinates
    constraining_left_o = np.zeros((int(len(DSB_coordinates)/2),1))
    constraining_right_o = np.zeros((int(len(DSB_coordinates)/2),1))
    #lists of initial gap sizes
    gapsize_left_o = np.zeros((int(len(DSB_coordinates)/2),1))
    gapsize_right_o = np.zeros((int(len(DSB_coordinates)/2),1))
    
    # parameters for taking snapshots
    steps_per_frame = 6 # frequency of taking snapshots
    extrusion_velocity = 1
    upper_time_limit = 60 # upper time limit for taking snapshots in minutes
    gapsize_xmin = 0
    
    preDSB_step_count = 0
    # Boolean flag as input for plotting function
    preDSB = True
    while preDSB_step_count<=preDSB_step_count_to_take_snapshots:
        # scan through all DSBs to check on the status of gap filling
        for i in range(int(len(DSB_coordinates)/2)):

            # identify the 2 adjacent BEs to the DSB
            test = np.asarray(boundary_coordinates) - DSB_coordinates[i*2]
            boundary_index = np.argmax(test>0)
            left_boundary = boundary_coordinates[boundary_index-1]
            right_boundary = boundary_coordinates[boundary_index]
            break_site = DSB_coordinates[i*2] # the left side of the DSB (DSB represented by 2 coordinates)
            
            # obtain the index of LEF that constrain the break site, if there's any
            test_left = left_extruders - break_site
            test_right = right_extruders - (break_site+1)
            test = np.multiply(test_left,test_right) 
            constraining_LEF_index = np.argwhere(test<0)
            # identify the inner most constraining LEF
            loop_size = np.abs(right_extruders[constraining_LEF_index] - left_extruders[constraining_LEF_index])
            if len(loop_size)>0:
                innermost_index = np.argmin(loop_size)

                constraining_left = left_extruders[constraining_LEF_index[innermost_index]] 
                constraining_right = right_extruders[constraining_LEF_index[innermost_index]] 

                if preDSB_step_count==0:
                    constraining_left_o[i] = constraining_left
                    constraining_right_o[i] = constraining_right
                    left_gap = np.ones(chrom_size)
                    right_gap = np.ones(chrom_size)
                    gapsize_left_o[i] = max(len(left_gap[int(constraining_left_o[i]):break_site]),len(left_gap[int(left_boundary):break_site]),300)
                    gapsize_right_o[i] =max(len(right_gap[(break_site+1)+1:int(constraining_right_o[i])]),len(right_gap[(break_site+1)+1:int(right_boundary)]),300)
            else:
                constraining_left = []
                constraining_right  = []
                if preDSB_step_count==0:
                    constraining_left_o[i] = left_boundary
                    constraining_right_o[i] = right_boundary
                    left_gap = np.ones(chrom_size)
                    right_gap = np.ones(chrom_size)
                    gapsize_left_o[i] = len(left_gap[int(constraining_left_o[i]):break_site])
                    gapsize_right_o[i] = len(right_gap[(break_site+1)+1:int(constraining_right_o[i])])
            if plot_snapshots:
                # take snap shots 
                unconstrained = False
                if i%10==0 and (preDSB_step_count%steps_per_frame==0):
                    synapsed = False
                    # create plotting folder if it doesn't exist
                    plotting_folder = output_folder+'/snapshots_sep{}_proc{}_superportion{}_superproc{}_ctcf{}_dsb{}_superloading{}_bs{}_DSB{}'.format(separations,processivity,longlived_fraction,processivity_longlived,BEstabilization_factor_o,DSBstabilization_factor,super_loading_factor, boundary_strength,i)
                    if not os.path.exists(plotting_folder):
                        os.makedirs(plotting_folder)
                    title = '{:.2f} minutes before DSB'.format((preDSB_step_count_to_take_snapshots-preDSB_step_count)/60*2) #asumming 1kb/s total extrusion speed
                    file_path = plotting_folder + '/step{}.jpeg'.format(preDSB_step_count)
                    plot_LEFs(file_path,processivity,left_extruders,right_extruders,left_boundary,right_boundary,constraining_left,constraining_right,constraining_left_o[i],constraining_right_o[i],break_site,title,super_extruder_len,gapsize_list_left[i//10],gapsize_list_right[i//10],gapsize_left_o[i],gapsize_right_o[i] , steps_per_frame,extrusion_velocity,upper_time_limit,unconstrained,synapsed,preDSB,gapsize_xmin)

        # take another step
        transloc.steps(1)
        preDSB_step_count += 1
        # update LEF positions
        super_left_extruders, super_right_extruders = transloc.getlonglivedLEFs()
        normal_left_extruders, normal_right_extruders = transloc.getLEFs()
        left_extruders  =  np.concatenate((super_left_extruders,normal_left_extruders))
        right_extruders  =  np.concatenate((super_right_extruders,normal_right_extruders))

    

    # list to store the maximum extrusion distance before each LEF unloads
    extruded_len_list = []

    # flag indicate whether the DSB is restrained at the time of DSB
    restrained_time0 = np.ones(int(len(DSB_coordinates)/2)) # fraction of DSBs constrained by LEFs at the time of the DSB
    restrained_realtime = np.ones(int(len(DSB_coordinates)/2)) # remaining fraction of DSBs constrained by LEFs (synapsed sites are considered constrained), updated throughout the simulation process
    restrained_realtime_old = np.copy(restrained_realtime)
    innerconstraining_LEF_size = np.zeros(int(len(DSB_coordinates)/2)) # size of innermost constraining LEF at each DSB site(if there's any)
    # array to store LEF identities at DSBs
    LEF_identity = np.zeros((5,int(len(DSB_coordinates)/2))) 
    # first row: constraining LEF identity: 0 if normal LEF, 1 if super LEF
    # 2nd row: number of normal LEFs in the left gap
    # 3rd row: number of longlived LEFs in the left gap
    # 4th row: number of normal LEFs in the right gap
    # 5th row: number of longlived LEFs in the right gap
    
    # time points for in silico ChIP
    chip_timepoints = [120, 300, 600, 1800, 2700, 3600] # 4min, 10min, 20min, 60min, 90min, 2hr (assuming 1kb/s extrusion speed)
    # array to store LEF coordiates across the indicated time points
    # odd rows for left motor subunits, and even rows for right motor subunits
    LEF_coordinates = np.zeros((14,len(left_extruders))) # number of LEFs in TADs when gap bridging finishes  
    portion_TADs_with_stabilized_LEFs = 0
    for i in range(int(len(boundary_coordinates)/2-1)):
        left_boundary = boundary_coordinates[i*2]
        right_boundary = boundary_coordinates[i*2+1]
        
        # record the fraction of TADs with stabilized LEFs
        if (left_boundary in left_extruders) or (right_boundary in right_extruders):
            portion_TADs_with_stabilized_LEFs += 1
        
    portion_TADs_with_stabilized_LEFs = portion_TADs_with_stabilized_LEFs/(len(boundary_coordinates)/2)
    LEF_coordinates[0,:] =  left_extruders
    LEF_coordinates[1,:] =  right_extruders
        
    # record the fraction of BE occupied by LEFs
    test_1 = np.intersect1d(left_extruders,boundary_coordinates[::2]) 
    test_2 = np.intersect1d(right_extruders,boundary_coordinates[1::2]) 
    portion_BE_occupied_by_LEF = (len(test_1) + len(test_2))/len(boundary_coordinates)
    
    overlap = np.intersect1d(np.concatenate([np.argwhere(left_extruders==test1) for test1 in test_1 if len(np.argwhere(left_extruders==test1))>0]),np.concatenate([np.argwhere(right_extruders==test2) for test2 in test_2 if len(np.argwhere(right_extruders==test2))>0]))
    portion_LEF_stabilized_by_BE = (len(test_1) + len(test_2)-len(overlap))/len(left_extruders) 
    
    # record the fraction of normal LEFs stabilized by BE
    test_1 = np.intersect1d(normal_left_extruders,boundary_coordinates[::2]) 
    test_2 = np.intersect1d(normal_right_extruders,boundary_coordinates[1::2]) 
    test_1_list = [np.argwhere(normal_left_extruders==test1) for test1 in test_1 if len(np.argwhere(normal_left_extruders==test1))>0]
    test_2_list = [np.argwhere(normal_right_extruders==test2) for test2 in test_2 if len(np.argwhere(normal_right_extruders==test2))>0]
    if len(test_1_list)>0:
        test_1_array = np.concatenate(test_1_list)
    if len(test_2_list)>0:
        test_2_array = np.concatenate(test_2_list)
    overlap = np.intersect1d(test_1_array,test_2_array)
    portion_normal_LEF_stabilized_by_BE = (len(test_1) + len(test_2)-len(overlap))/len(normal_left_extruders) 
    
    # record the fraction of long-lived LEFs stabilized by BE
    test_1 = np.intersect1d(super_left_extruders,boundary_coordinates[::2]) 
    test_2 = np.intersect1d(super_right_extruders,boundary_coordinates[1::2]) 
    test_1_list = [np.argwhere(super_left_extruders==test1) for test1 in test_1 if len(np.argwhere(super_left_extruders==test1))>0]
    test_2_list = [np.argwhere(super_right_extruders==test2) for test2 in test_2 if len(np.argwhere(super_right_extruders==test2))>0]
    if len(test_1_list)>0:
        test_1_array = np.concatenate(test_1_list)
    if len(test_2_list)>0:
        test_2_array = np.concatenate(test_2_list)
    overlap = np.intersect1d(test_1_array,test_2_array)
    if len(super_left_extruders)>0:
        portion_super_LEF_stabilized_by_BE = (len(test_1) + len(test_2)-len(overlap))/len(super_left_extruders) 
    else:
        portion_super_LEF_stabilized_by_BE = np.nan 
    
    stabilized_fraction.append([portion_BE_occupied_by_LEF*100,
                                portion_TADs_with_stabilized_LEFs*100,
                              portion_LEF_stabilized_by_BE*100,
                              portion_normal_LEF_stabilized_by_BE,
                              portion_super_LEF_stabilized_by_BE])
    
    stabilized_df = pd.DataFrame(stabilized_fraction, 
             columns=['percent BE occupied by LEF ',
                     'percent TADs with stabilized LEF',
                     'percent LEF stabilized by BE',
                     'percent normal LEF stabilized by BE',
                     'percent super LEF stabilized by BE']) 
    
    stabilized_df.to_csv(output_folder+'/stabilized_df_sep{}_proc{}_superportion{}_superproc{}_ctcf{}_dsb{}_superloading{}_bs{}.csv'.format(separations,processivity,longlived_fraction,processivity_longlived,BEstabilization_factor_o,DSBstabilization_factor,super_loading_factor, boundary_strength))

    
    # counter for steps taken since the introduction of DSB    
    step_count = 0

    #introduce the DSB
    preDSB = False
    # make double strand break site impermeable
    stall_prob_left[DSB_coordinates[1::2]] = 1 # even elements of the DSB coordinates
    stall_prob_right[DSB_coordinates[::2]] = 1 # odd elements of the DSB coordinates
    # boosting by DSB
    death_prob[DSB_coordinates] =1. / (0.5*processivity/(LEF_step_probability))/ DSBstabilization_factor
    death_prob_longlived[DSB_coordinates] =1. / (0.5*processivity_longlived/(LEF_step_probability))/ DSBstabilization_factor

    # DSB flag to ensure correct valency at DSB site: DSB can only stabilize one LEF
    DSB_flag = np.zeros(chrom_size, int)
    DSB_flag[DSB_coordinates] = 1 
    transloc.updateStallprob(stall_prob_left,stall_prob_right)
    transloc.updateDSBFlag(DSB_flag)
    # implement targeted loading mechanism
    loading_prob[DSB_coordinates]= 1/chrom_size*super_loading_factor
    transloc.updateEmissionProb(loading_prob)

    #flags to indicate gap exists at different threshold level for a subTAD in a given tile
    gap_threshold_flag = np.ones((int(len(DSB_coordinates)/2),len(alpha_list)))

    
    while np.count_nonzero(gap_threshold_flag)>0 and np.count_nonzero(restrained_realtime)>0 and step_count<step_count_limit:
        
        # in silico ChIP experiment at specified time points
        if step_count in chip_timepoints:
                row_num = (np.where(np.asarray(chip_timepoints)==step_count)[0]+1)*2
                LEF_coordinates[row_num,:]=left_extruders
                LEF_coordinates[row_num+1,:]=right_extruders
                
         # remove half of gap sizes when reaching upper_time_limit
        if (step_count*2/extrusion_velocity/60)%(upper_time_limit/2)==0 and (step_count*2/extrusion_velocity/60)>= upper_time_limit:
            for p in range(len(gapsize_list_left)):
                del gapsize_list_left[p][0:int((upper_time_limit/2)*60*extrusion_velocity/2/steps_per_frame)]
                del gapsize_list_right[p][0:int((upper_time_limit/2)*60*extrusion_velocity/2/steps_per_frame)]
            gapsize_xmin += upper_time_limit/2
        #scan through all the subTADs to check on the status of gap filling
        for i in range(int(len(DSB_coordinates)/2)):

            test = np.asarray(boundary_coordinates) - DSB_coordinates[i*2]
            boundary_index = np.argmax(test>0)
            left_boundary = boundary_coordinates[boundary_index-1]
            right_boundary = boundary_coordinates[boundary_index]
            break_site = DSB_coordinates[i*2] # the left side of the DSB (DSB represented by 2 coordinates)
            

            # obtain the index of LEF that constrain the DSB site, if there's any
            test_left = left_extruders - break_site
            test_right = right_extruders - (break_site+1)
            test = np.multiply(test_left,test_right) 
            constraining_LEF_index = np.argwhere(test<0)

            # check if two DSB ends are restrained and that there's gap with the specific threshold
            if len(constraining_LEF_index)==0 and np.count_nonzero(gap_threshold_flag[i,:])>0:
                restrained_realtime[i]=0
                
                if step_count == 0:
                    restrained_time0[i]=0
                    fail_gap_length_l+=[0]
                elif restrained_realtime_old[i] == 1:
                    fail_gap_length_l+=[innerconstraining_LEF_size[i]]
            
            if plot_snapshots:
                if i%10==0:
                    if restrained_realtime_old[i]==1 and restrained_realtime[i]==0:
                        unconstrained = True
                        synapsed = False
                        title = '{:.2f} minutes after DSB'.format(step_count/60*2) #asumming 1kb/s total extrusion speed
                        file_path = plotting_folder + '/step{}.jpeg'.format(step_count+preDSB_step_count_to_take_snapshots+1)
                        if unconstrained:
                            constraining_left = []
                            constraining_right = []
                        plot_LEFs(file_path,processivity,left_extruders,right_extruders,left_boundary,right_boundary,constraining_left,constraining_right,constraining_left_o[i],constraining_right_o[i],break_site,title,super_extruder_len,gapsize_list_left[i//10],gapsize_list_right[i//10],gapsize_left_o[i],gapsize_right_o[i] , steps_per_frame,extrusion_velocity,upper_time_limit,unconstrained,synapsed,preDSB,gapsize_xmin)


            if restrained_realtime[i]==1: 
                unconstrained = False
                if gap_threshold_flag[i,0]==1: # if the most stringent synapsis criteria is not yet met
                    # identify the inner most constraining LEF
                    loop_size = np.abs(right_extruders[constraining_LEF_index] - left_extruders[constraining_LEF_index])
                    innermost_index = np.argmin(loop_size)

                    constraining_left = left_extruders[constraining_LEF_index[innermost_index]] 
                    constraining_right = right_extruders[constraining_LEF_index[innermost_index]] 
                    innerconstraining_LEF_size[i] = constraining_right-constraining_left

                    left_gap = np.ones(chrom_size)
                    right_gap = np.ones(chrom_size)
                    unfilled_gap_left = len(left_gap[constraining_left[0]+1:break_site])
                    unfilled_gap_right = len(right_gap[(break_site+1)+1:constraining_right[0]])

                    # test for the left side of the break
                    # find the LEFs between the left foot of the innermost restraing LEF and the break site
                    test_left_1 = np.where((left_extruders-constraining_left)>=0,1,0) 
                    test_left_2 = np.where((right_extruders-break_site)<=0,1,0)
                    test_left = np.multiply(test_left_1,test_left_2) 
                    left_between_index = np.argwhere(test_left==1)
                    # continue to test for the right side of the break
                    # find the LEFs between the break site and the right foot of the innermost restraing LEF 
                    test_right_1 = np.where((left_extruders-(break_site+1))>=0,1,0) 
                    test_right_2 = np.where((right_extruders-constraining_right)<=0,1,0)
                    test_right = np.multiply(test_right_1,test_right_2) 
                    right_between_index = np.argwhere(test_right==1)

                    if len(left_between_index)>0:

                        for j in range(len(left_between_index)):
                            left_gap[left_extruders[left_between_index[j]][0]:(right_extruders[left_between_index[j]][0]+1)]=0

                        unfilled_gap_left = np.count_nonzero(left_gap[constraining_left[0]+1:break_site])

                    if len(right_between_index)>0:

                        for j in range(len(right_between_index)):
                            right_gap[left_extruders[right_between_index[j]][0]:(right_extruders[right_between_index[j]][0]+1)]=0

                        unfilled_gap_right = np.count_nonzero(right_gap[(break_site+1)+1:constraining_right[0]])

                    if plot_snapshots:
                        #take snap shots (randomly select ~20 DSB sites for snapshots every steps_per_frame steps) 
                        synapsed = False
                        if unfilled_gap_left <= alpha_list[0]*2 and unfilled_gap_right <= alpha_list[0]*2:
                            synapsed = True
                        if i%10==0 and (step_count%steps_per_frame==0 or synapsed):

                            gapsize_list_left[i//10]+=[unfilled_gap_left]
                            gapsize_list_right[i//10]+=[unfilled_gap_right]
                            # create plotting folder if it doesn't exist
                            plotting_folder = output_folder+'/snapshots_sep{}_proc{}_superportion{}_superproc{}_ctcf{}_dsb{}_superloading{}_bs{}_DSB{}'.format(separations,processivity,longlived_fraction,processivity_longlived,BEstabilization_factor_o,DSBstabilization_factor,super_loading_factor, boundary_strength,i)
                            if not os.path.exists(plotting_folder):
                                os.makedirs(plotting_folder)
                            title = '{:.2f} minutes after DSB'.format(step_count/60*2) #asumming 1kb/s total extrusion speed
                            file_path = plotting_folder + '/step{}.jpeg'.format(step_count+preDSB_step_count_to_take_snapshots+1)
                            plot_LEFs(file_path,processivity,left_extruders,right_extruders,left_boundary,right_boundary,constraining_left,constraining_right,constraining_left_o[i],constraining_right_o[i],break_site,title,super_extruder_len,gapsize_list_left[i//10],gapsize_list_right[i//10],gapsize_left_o[i],gapsize_right_o[i] , steps_per_frame,extrusion_velocity,upper_time_limit,unconstrained,synapsed,preDSB,gapsize_xmin)

                    for k in range(len(alpha_list)):
                        # check if the gap has been filled on this threshold level
                        if gap_threshold_flag[i,k]==1:

                            if unfilled_gap_left <= alpha_list[k]*2 and unfilled_gap_right <= alpha_list[k]*2: # To speed up computation, only look at the gap on the right if the gap on the left meets threshold

                                # record the number of LEFs in each DSB-induced sub_TAD
                                # find the LEFs between the left_boundary and the right_boundary
                                test_1 = np.where((left_extruders-left_boundary)*(left_extruders-right_boundary)<=0,1,0) 
                                test_2 = np.where((right_extruders-left_boundary)*(right_extruders-right_boundary)<=0,1,0) 
                                test_binary = np.multiply(test_1,test_2) 
                                between_index = np.argwhere(test_binary==1)
                                num_LEFs_between = np.count_nonzero(test_1)+np.count_nonzero(test_2)-len(between_index)

                                # obtain further info on the DSB
                                if i == 0:
                                    dist_to_neighbor_DSB = DSB_coordinates[(i+1)*2] - break_site
                                elif i==len(DSB_coordinates)/2-1:
                                    dist_to_neighbor_DSB = break_site - DSB_coordinates[(i-1)*2]
                                else:
                                    dist_to_neighbor_DSB = DSB_coordinates[(i+1)*2] - DSB_coordinates[(i-1)*2]

                                # record LEF identity
                                if constraining_right[0] in super_right_extruders:
                                    LEF_identity[0,i] = 1
                                # find the normal LEFs in the left gap 
                                test_1 = np.where((normal_left_extruders-constraining_left)>=0,1,0) 
                                test_2 = np.where((normal_right_extruders-break_site)<=0,1,0)
                                test = np.multiply(test_1,test_2) 
                                left_between_normal_index = np.argwhere(test==1)
                                LEF_identity[1,i] = len(left_between_normal_index)
                                # find the longlived LEFs in the left gap 
                                test_1 = np.where((super_left_extruders-constraining_left)>=0,1,0) 
                                test_2 = np.where((super_right_extruders-break_site)<=0,1,0)
                                test = np.multiply(test_1,test_2) 
                                left_between_super_index = np.argwhere(test==1)
                                LEF_identity[2,i] = len(left_between_super_index)
                                # find the normal LEFs in the right gap 
                                test_1 = np.where((normal_left_extruders-(break_site+1))>=0,1,0) 
                                test_2 = np.where((normal_right_extruders-constraining_right)<=0,1,0)
                                test = np.multiply(test_1,test_2) 
                                right_between_normal_index = np.argwhere(test==1)
                                LEF_identity[3,i] = len(right_between_normal_index)
                                # find the longlived LEFs in the right gap 
                                test_1 = np.where((super_left_extruders-(break_site+1))>=0,1,0) 
                                test_2 = np.where((super_right_extruders-constraining_right)<=0,1,0)
                                test = np.multiply(test_1,test_2) 
                                right_between_super_index = np.argwhere(test==1)
                                LEF_identity[4,i] = len(right_between_super_index)

                                dist_to_CTCF = break_site - left_boundary
                                subTAD_size = right_boundary - left_boundary

                                first_passage.append([separations,
                                                      processivity,
                                                      boundary_strength,
                                                      subTAD_size,
                                                      alpha_list[k],
                                                      step_count,
                                                    num_LEFs_between,
                                                    dist_to_neighbor_DSB,
                                                     dist_to_CTCF])

                                gap_threshold_flag[i,k] = 0
                                if k==0:
                                    success_gap_length_l += [innerconstraining_LEF_size[i]]


        # take another step
        transloc.steps(1)
        step_count += 1
        #time_recorder += 1
        # store the position of left motor subunits
        old_left_extruders = left_extruders
        old_right_extruders = right_extruders
        #update LEF positions
        super_left_extruders, super_right_extruders = transloc.getlonglivedLEFs()
        normal_left_extruders, normal_right_extruders = transloc.getLEFs()
        left_extruders  =  np.concatenate((super_left_extruders,normal_left_extruders))
        right_extruders  =  np.concatenate((super_right_extruders,normal_right_extruders))

        # identify the extruders that have been unloaded
        diff = abs(old_left_extruders-left_extruders)
        unloaded_index = np.argwhere(diff>1)
        extruded_len =[old_right_extruders[i]-old_left_extruders[i] for i in range(len(unloaded_index))]
        extruded_len_list += extruded_len
        
        restrained_realtime_old = np.copy(restrained_realtime)
        
    # reverse the DSB for the next round of simulation (not necessary for parallel implementation)
    stall_prob_left[DSB_coordinates] = 0
    stall_prob_right[DSB_coordinates] = 0
    
    death_prob = np.zeros(chrom_size, dtype=np.double) + 1. / (0.5*processivity/(LEF_step_probability)) 
    death_prob_longlived = np.zeros(chrom_size, dtype=np.double) + 1. / (0.5*processivity_longlived/(LEF_step_probability)) 
    
    DSB_flag = np.zeros(chrom_size, int)
    loading_prob = np.ones(chrom_size)/chrom_size
    
    # record simulation data associated with a specific distance threshold alpha (note some of these are independent of alpha)
    for k in range(len(alpha_list)):
        success_index = np.nonzero(gap_threshold_flag[:,k]-1) # index of the DSBs synapsed at this distance threshold
        restrained_time0_index = np.nonzero(restrained_time0[:]) # index of DSB constrained at the time of DSB
        restrained_index = np.nonzero(restrained_realtime[:]) # index of remaining DSB that are constrained, updated throughout simulation
        success_TAD_sizes = DSB_TAD_array_size[success_index] # array of TAD sizes containing synapsed DSB
        restrained_time0_TAD_sizes = DSB_TAD_array_size[restrained_time0_index] # array of TAD sizes containing DSBs that are constrained at the time of DSB induction
        restrained_TAD_sizes = DSB_TAD_array_size[restrained_index] # array of TAD sizes containing DSBs that are synapsed or remained constrained throughout simulation
        DSB_count = np.count_nonzero(DSB_TAD_array_size) # total number of DSB sites
        success_200kb = len(np.argwhere(success_TAD_sizes==200)) # number of 200kb TADs containing synapsed DSB
        success_400kb = len(np.argwhere(success_TAD_sizes==400)) # number of 400kb TADs containing synapsed DSB
        success_800kb = len(np.argwhere(success_TAD_sizes==800)) # number of 800kb TADs containing synapsed DSB
        success_1200kb = len(np.argwhere(success_TAD_sizes==1200)) # number of 1200kb TADs containing synapsed DSB
        restrained_time0_200kb = len(np.argwhere(restrained_time0_TAD_sizes==200)) # number of 200kb TADs containing whose DSB is constrained when induced
        restrained_time0_400kb = len(np.argwhere(restrained_time0_TAD_sizes==400)) # number of 400kb TADs containing whose DSB is constrained when induced
        restrained_time0_800kb = len(np.argwhere(restrained_time0_TAD_sizes==800)) # number of 800kb TADs containing whose DSB is constrained when induced
        restrained_time0_1200kb = len(np.argwhere(restrained_time0_TAD_sizes==1200)) # number of 1200kb TADs containing whose DSB is constrained when induced
        restrained_200kb = len(np.argwhere(restrained_TAD_sizes==200)) # number of 200kb TADs containing whose DSB is synapsed or remained constrained throughout simulation
        restrained_400kb = len(np.argwhere(restrained_TAD_sizes==400)) # number of 400kb TADs containing whose DSB is synapsed or remained constrained throughout simulation
        restrained_800kb = len(np.argwhere(restrained_TAD_sizes==800)) # number of 800kb TADs containing whose DSB is synapsed or remained constrained throughout simulation
        restrained_1200kb = len(np.argwhere(restrained_TAD_sizes==1200)) # number of 1200kb TADs containing whose DSB is synapsed or remained constrained throughout simulation
        restrained_fraction.append([separations,
                                    processivity,
                                    boundary_strength,
                                    alpha_list[k], 
                                    np.count_nonzero(restrained_time0[:])/DSB_count,
                                    np.count_nonzero(restrained_realtime[:])/DSB_count,
                                    len(success_index[0]),
                                    len(success_index[0])/ DSB_count,
                                    restrained_time0_200kb,
                                    restrained_time0_400kb,
                                    restrained_time0_800kb,
                                    restrained_time0_1200kb,
                                    restrained_200kb,
                                    restrained_400kb,
                                    restrained_800kb,
                                    restrained_1200kb,
                                    success_200kb,
                                    success_400kb,
                                    success_800kb,
                                    success_1200kb,
                                    np.mean(extruded_len_list),
                                     portion_BE_occupied_by_LEF*100,
                                     portion_TADs_with_stabilized_LEFs*100,
                                     portion_LEF_stabilized_by_BE*100])
        

    restrained_df = pd.DataFrame(restrained_fraction, 
             columns=['separations',
                      'processivity',
                      'boundary strength',
                      'threshold',
                      'restrained proportion time0',
                    'restrained proportion realtime',
                     'successful repair count',
                     'repaired proportion',
                      'initial restrained 200kb count',
                     'initial restrained 400kb count',
                     'initial restrained 800kb count',
                     'initial restrained 1200kb count',
                      'realtime restrained 200kb count',
                     'realtime restrained 400kb count',
                     'realtime restrained 800kb count',
                     'realtime restrained 1200kb count',
                    'repaired 200kb count',
                     'repaired 400kb count',
                     'repaired 800kb count',
                     'repaired 1200kb count',
                     'average extruded length',
                     'portion_BE_occupied_by_LEF',
                     'portion_TADs_with_stabilized_LEFs',
                     'portion_LEF_stabilized_by_BE']) 



    stats_df = pd.DataFrame(first_passage, 
                     columns=['separations',
                              'processivity',
                              'boundary_strength',
                              'TAD_size',
                              'threshold',
                              'first_passage_time',
                             'subTAD_LEFs_num',
                             'dist_to_neighbor_DSB',
                             'dist_to_CTCF']) 


    restrained_df.to_csv(output_folder+'/restrained_df_sep{}_proc{}_superportion{}_superproc{}_ctcf{}_dsb{}_superloading{}_bs{}.csv'.format(separations,processivity,longlived_fraction,processivity_longlived,BEstabilization_factor_o,DSBstabilization_factor,super_loading_factor, boundary_strength))
    stats_df.to_csv(output_folder+'/stats_df_sep{}_proc{}_superportion{}_superproc{}_ctcf{}_dsb{}_superloading{}_bs{}.csv'.format(separations,processivity,longlived_fraction,processivity_longlived,BEstabilization_factor_o,DSBstabilization_factor,super_loading_factor,boundary_strength))
    
    file = output_folder+'/GapLength_sep{}_proc{}_superportion{}_superproc{}_ctcf{}_dsb{}_superloading{}_bs{}.npy'.format(separations,processivity,longlived_fraction,processivity_longlived,BEstabilization_factor_o,DSBstabilization_factor,super_loading_factor, boundary_strength)
    # saving gap lengths
    with open(file, 'wb') as g:
        np.save(g, np.asarray(fail_gap_length_l))
        np.save(g, np.asarray(success_gap_length_l))
        
    # save the distribution of LEFs in different TAD sizes
    file = output_folder+'/LEFcoordinates_sep{}_proc{}_superportion{}_superproc{}_ctcf{}_dsb{}_superloading{}_bs{}.npy'.format(separations,processivity,longlived_fraction,processivity_longlived,BEstabilization_factor_o,DSBstabilization_factor,super_loading_factor,boundary_strength)
    # saving LEF counts
    with open(file, 'wb') as g:
        np.save(g, LEF_coordinates)
    
    file = output_folder+'/LEFidentity_sep{}_proc{}_superportion{}_superproc{}_ctcf{}_dsb{}_superloading{}_bs{}.npy'.format(separations,processivity,longlived_fraction,processivity_longlived,BEstabilization_factor_o,DSBstabilization_factor,super_loading_factor,boundary_strength)
    # saving gap lengths
    with open(file, 'wb') as g:
        np.save(g, LEF_identity)
    
    return None

   

    


## Run the parallel processing

In [ ]:
from itertools import product
num_cores = 30

generator_of_inputs = product(sep_proc_list, boundary_strength_list,BE_stabilization_factor_list, DSB_stabilization_factor_list,longlived_fraction_list,targetedloading_factor_list)

# create independent copies of the shared parameters
inputs = [(a,b,c,d,e,f,proc_ratio, chrom_size,step_count_limit,alpha_list,boundary_coordinates,DSB_coordinates,LEF_step_probability,DSB_TAD_array_size, plot_snapshots) for a,b,c,d,e,f in generator_of_inputs]

pool = Pool(num_cores)

results = pool.starmap(do_one_parameter_set, inputs)
